In [1]:
# Import Github repo
from google.colab import files
import zipfile
import os

print('Upload project file')
uploaded = files.upload()

zip = list(uploaded.keys())[0]
with zipfile.ZipFile(zip, 'r') as zip_ref:
  zip_ref.extractall('/content')

print(f'Extracted {zip}')
print('Availale Files/Folders')
for item in os.listdir('/content'):
  print(f'{item}')

Upload project file


Saving NNIK.zip to NNIK.zip
Extracted NNIK.zip
Availale Files/Folders
.config
NNIK.zip
NNIK
sample_data


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Setup Paths
import sys
from pathlib import Path

root = Path('/content/NNIK') if Path('/content/NNIK').exists() else Path('/content/NNIK-main')
print(f'Project root - {root}')
print(f'Project exists - {root.exists()}')

remove = [p for p in sys.path if 'NNIK' in p or 'Scripts' in p]
for path in remove:
  sys.path.remove(path)

project_paths = [
    str(root),
    str(root / 'Scripts'),
    str(root / 'Scripts' / 'Models'),
    str(root / 'Scripts' / 'Models' / 'Machine_Learning'),
    str(root / 'Scripts' / 'Models' / 'Traditional'),
]

for path in project_paths:
  if path not in sys.path:
    sys.path.insert(0, path)

print('Paths configured')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Project root - /content/NNIK
Project exists - True
Paths configured


In [4]:
# Install dependencies
!pip install -q torch torchvision torchaudio scikit-learn pandas matplotlib numpy tqdm pyyaml scipy

In [5]:
# Import project modules
import importlib.util
import traceback
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import yaml
from datetime import datetime

def import_path(module_name, file_path):
  try:
    if not file_path.exists():
      print(f'File not found - {file_path}')
      return None

    spec = importlib.util.spec_from_file_location(module_name, file_path)
    if spec is None or spec.loader is None:
      print(f'Couldnt create spec for {module_name}')
      return None

    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)
    return module
  except Exception as e:
    print(f'Error importing {module_name} - {str(e)}')
    return None

utils_module = import_path('utils', root / 'Scripts' / 'utils.py')
data_gen_module = import_path('data_gen', root / 'Scripts' / 'data_gen.py')
training_module = import_path('training', root / 'Scripts' / 'training.py')
testing_module = import_path('testing', root / 'Scripts' / 'testing.py')

if utils_module:
    print('utils.py imported')
if data_gen_module:
    print('data_gen.py imported')
if training_module:
    print('training.py imported')
if testing_module:
    print('testing.py imported')

utils.py imported
data_gen.py imported
training.py imported
testing.py imported


In [6]:
# Quick test
data_directory = root / 'data'
print(f'data directory - {data_directory}')
print(f'data directory exists - {data_directory.exists()}')

if data_directory.exists():
  print('\nTraining data - ')
  training_directory = data_directory / 'Training'
  if training_directory.exists():
    training_files = list(training_directory.glob('*.json'))
    print(f'Found {len(training_files)} training files')
    for f in sorted(training_files)[:5]:
      print(f.name)
  print('\nTesting data:')
  testing_directory = data_directory / 'Testing'
  if testing_directory.exists():
    testing_files = list(testing_directory.glob('*.json'))
    print(f'Found {len(testing_files)} testing files')
    for f in sorted(testing_files)[:5]:
      print(f.name)



data directory - /content/NNIK/data
data directory exists - True

Training data - 
Found 16 training files
10_training.json
10_training_solutions.json
3_training.json
3_training_solutions.json
4_training.json

Testing data:
Found 16 testing files
10_testing.json
10_testing_solutions.json
3_testing.json
3_testing_solutions.json
4_testing.json


In [7]:
# Training
from Scripts.training import load_ik_data, train_all_models, setup_gpu
from Scripts.testing import create_models
import numpy as np
import pandas as pd
import torch

dof_range = [3, 4, 5, 6, 7, 8, 9, 10]
sample_limit = None

device = setup_gpu()


all_trained_models = {}
all_training_results = []

for dof in dof_range:
    print(f'\nDOF={dof} training')

    train_poses = data_directory / 'Training' / f'{dof}_training.json'
    train_solutions = data_directory / 'Training' / f'{dof}_training_solutions.json'

    if not (train_poses.exists() and train_solutions.exists()):
        print(f'no data for DOF={dof}')
        continue

    X_train, y_train = load_ik_data(train_poses, train_solutions)

    if sample_limit and len(X_train) > sample_limit:
        idx = np.random.choice(len(X_train), sample_limit, replace=False)
        X_train, y_train = X_train[idx], y_train[idx]

    print(f'Training data - {X_train.shape} -> {y_train.shape}')

    models_for_dof = create_models(input_dim=X_train.shape[1], output_dim=y_train.shape[1])

    selected_models = {}
    # Remove GPR for now - it's too slow with 2000 samples
    for name in ['ANN', 'KNN', 'ELM', 'RandomForest', 'SVM', 'MDN', 'CVAE']:  # Removed GPR
        if name in models_for_dof and models_for_dof[name] is not None:
            selected_models[name] = models_for_dof[name]

    print(f'models: {list(selected_models.keys())}')

    trained_models = train_all_models(selected_models, X_train, y_train, use_parallel=True)

    all_trained_models[dof] = trained_models

    for name, result in trained_models.items():
        if 'error' not in result:
            all_training_results.append({
                'dof': dof,
                'model': name,
                'training_time': result['training_time'],
                'samples': len(X_train),
                'status': 'success'
            })
        else:
            all_training_results.append({
                'dof': dof,
                'model': name,
                'training_time': 0,
                'samples': len(X_train),
                'status': f'failed: {result['error'][:50]}'
            })

    print(f'\nDOF={dof} training complete - {len([r for r in trained_models.values() if 'error' not in r])} models trained')

print(f'\nTraining Complete')

training_df = pd.DataFrame(all_training_results)
if not training_df.empty:
    print('Training Summary:')
    print(training_df)

    successful = training_df[training_df['status'] == 'success']
    if not successful.empty:
        print(f'\nStatistics:')
        pivot = successful.pivot_table(values='training_time', index='model', columns='dof', aggfunc='mean')
        print(pivot.round(2))

print(f'\nTotal trained models: {sum(len([m for m in models.values() if 'error' not in m]) for models in all_trained_models.values())}')

GPU: Tesla T4 (15.8 GB)

DOF=3 training
Loaded data: X shape = (2000, 6), y shape = (2000, 3)
DOF from data: 3
Training data - (2000, 6) -> (2000, 3)
models: ['ANN', 'KNN', 'ELM', 'RandomForest', 'SVM', 'MDN', 'CVAE']
GPU: Tesla T4 (15.8 GB)

Training 4 CPU models in parallel...
Training KNN...
Training ELM...
Training RandomForest...
Training SVM...
  ✓ KNN completed in 0.02s
  ✓ ELM completed in 0.25s
  ✓ RandomForest completed in 0.50s
  ✓ SVM completed in 0.71s

Training 3 GPU models...
Training ANN...
  Using GPU for ANN
Epoch [20/20], Train Loss: 0.5440, Val Loss: 0.4884
  ✓ ANN completed in 7.04s
Training MDN...
  Using GPU for MDN
Epoch [20/50], Loss: -0.4401
Epoch [40/50], Loss: -1.6126
  ✓ MDN completed in 14.99s
Training CVAE...
  Using GPU for CVAE
Epoch [20/50], Loss: 0.9036
Epoch [40/50], Loss: 0.5735
  ✓ CVAE completed in 9.81s

GPU Memory: 0.02 GB allocated

DOF=3 training complete - 7 models trained

DOF=4 training
Loaded data: X shape = (2000, 6), y shape = (2000, 4)


In [8]:
# Testing
from Scripts.testing import evaluate_all_models, create_results_dataframe

all_test_results = []

print('\n')
print('Testing')

for dof, trained_models in all_trained_models.items():
    if not trained_models:
        continue

    print(f'\nDOF={dof} testing')

    test_poses = data_directory / 'Testing' / f'{dof}_testing.json'
    test_solutions = data_directory / 'Testing' / f'{dof}_testing_solutions.json'

    if not (test_poses.exists() and test_solutions.exists()):
        print(f'testing data not found for dof={dof}')
        continue

    X_test, y_test = load_ik_data(test_poses, test_solutions)

    if sample_limit and len(X_test) > sample_limit//2:
        idx = np.random.choice(len(X_test), sample_limit//2, replace=False)
        X_test, y_test = X_test[idx], y_test[idx]

    print(f'Testing data: {X_test.shape} -> {y_test.shape}')

    evaluation_results = evaluate_all_models(trained_models, X_test, y_test, force_cpu=False)

    for name, results in evaluation_results.items():
        if 'error' not in results:
            all_test_results.append({
                'dof': dof,
                'model': name,
                'position_rmse': results['position_rmse'],
                'joint_rmse': results['joint_rmse'],
                'training_time': results['training_time'],
                'inference_time': results['inference_time'],
                'inference_time_per_sample': results['inference_time_per_sample'],
                'test_samples': len(X_test),
                'status': 'success'
            })

    print(f'\ndof={dof} testing complete')

print(f'\ntesting complete')
print('='*60)

if all_test_results:
    results_df = pd.DataFrame(all_test_results)

    print('Complete Results:')
    display_cols = ['dof', 'model', 'joint_rmse', 'training_time', 'inference_time_per_sample']
    print(results_df[display_cols].round(4))

    results_path = root / 'results'
    results_path.mkdir(exist_ok=True)
    results_df.to_csv(results_path / 'explicit_training_results.csv', index=False)
    print(f'\nResults saved to: {results_path / 'explicit_training_results.csv'}')

    print(f'\nModel Performance Summary (Average across DOFs):')
    model_summary = results_df.groupby('model').agg({
        'joint_rmse': 'mean',
        'training_time': 'mean',
        'inference_time_per_sample': 'mean'
    }).round(4)
    print(model_summary)

else:
    print('No test results available')



Testing

DOF=3 testing
Loaded data: X shape = (1000, 6), y shape = (1000, 3)
DOF from data: 3
Testing data: (1000, 6) -> (1000, 3)
Evaluating KNN...
  ✓ Joint RMSE: 1.0648, Inference: 0.008s
Evaluating ELM...
  ✓ Joint RMSE: 1.3555, Inference: 0.000s
Evaluating RandomForest...
  ✓ Joint RMSE: 0.8570, Inference: 0.013s
Evaluating SVM...
  ✓ Joint RMSE: 1.1026, Inference: 0.127s
Evaluating ANN...
  ✓ Joint RMSE: 1.3034, Inference: 0.001s
Evaluating MDN...
  ✓ Joint RMSE: 1.3428, Inference: 0.001s
Evaluating CVAE...
  ✓ Joint RMSE: 0.9618, Inference: 0.005s

dof=3 testing complete

DOF=4 testing
Loaded data: X shape = (1000, 6), y shape = (1000, 4)
DOF from data: 4
Testing data: (1000, 6) -> (1000, 4)
Evaluating KNN...
  ✓ Joint RMSE: 1.4249, Inference: 0.010s
Evaluating ELM...
  ✓ Joint RMSE: 1.5366, Inference: 0.000s
Evaluating RandomForest...
  ✓ Joint RMSE: 1.3798, Inference: 0.013s
Evaluating SVM...
  ✓ Joint RMSE: 1.4019, Inference: 0.266s
Evaluating ANN...
  ✓ Joint RMSE: 1.4887,

In [ ]:
# Visualization and Analysis
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap
from scipy.spatial import ConvexHull
from scipy.interpolate import griddata
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# Setup consistent visual style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

def colors(results_df):
    # Create consistent color mapping for models
    unique_models = results_df['model'].unique()
    return dict(zip(unique_models, sns.color_palette('husl', len(unique_models))))

def accuracy_vs_speed(results_df, ax, model_colors):
    # Plot accuracy vs speed
    for model in results_df['model'].unique():
        model_data = results_df[results_df['model'] == model]
        avg_acc = model_data['joint_rmse'].mean()
        avg_time = model_data['training_time'].mean()
        ax.scatter(avg_time, avg_acc, s=120, alpha=0.8,
                  label=model, color=model_colors[model], edgecolors='white', linewidth=1)
        ax.annotate(model, (avg_time, avg_acc), xytext=(8, 8),
                   textcoords='offset points', fontsize=9, fontweight='bold')

    ax.set_xlabel('Average Training Time (s)', fontsize=11, fontweight='bold')
    ax.set_ylabel('Average Joint RMSE', fontsize=11, fontweight='bold')
    ax.set_title('Accuracy vs Speed Tradeoff', fontsize=12, fontweight='bold', pad=20)
    ax.set_xscale('log')
    ax.grid(True, alpha=0.3)

def heatmap(results_df, ax):
    # Plot model performance heatmap with improved colormap
    pivot_rmse = results_df.pivot_table(values='joint_rmse', index='model', columns='dof', aggfunc='mean')

    # Use perceptually uniform colormap
    im = ax.imshow(pivot_rmse.values, cmap='viridis', aspect='auto', interpolation='bilinear')
    ax.set_xticks(range(len(pivot_rmse.columns)))
    ax.set_xticklabels(pivot_rmse.columns, fontweight='bold')
    ax.set_yticks(range(len(pivot_rmse.index)))
    ax.set_yticklabels(pivot_rmse.index, fontweight='bold')
    ax.set_xlabel('DOF', fontsize=11, fontweight='bold')
    ax.set_ylabel('Model', fontsize=11, fontweight='bold')
    ax.set_title('Joint RMSE Heatmap', fontsize=12, fontweight='bold', pad=20)

    # Add colorbar with proper positioning
    cbar = plt.colorbar(im, ax=ax, shrink=0.8)
    cbar.set_label('Joint RMSE', fontweight='bold')

def training_distribution(results_df, ax):
    # Plot training time distribution
    training_times = results_df.groupby('model')['training_time'].mean().sort_values()
    colors = ['green' if t < 10 else 'orange' if t < 30 else 'red' for t in training_times.values]

    bars = ax.bar(range(len(training_times)), training_times.values, color=colors, alpha=0.8, edgecolor='white')
    ax.set_xticks(range(len(training_times)))
    ax.set_xticklabels(training_times.index, rotation=45, ha='right', fontweight='bold')
    ax.set_ylabel('Training Time (s)', fontsize=11, fontweight='bold')
    ax.set_title('Average Training Time by Model', fontsize=12, fontweight='bold', pad=20)
    ax.set_yscale('log')
    ax.grid(True, alpha=0.3)

    # Add value labels
    for bar, val in zip(bars, training_times.values):
        ax.text(bar.get_x() + bar.get_width()/2., bar.get_height(),
               f'{val:.1f}s', ha='center', va='bottom', fontsize=9, fontweight='bold')

def speed_comparison(results_df, ax):
    # Plot inference speed comparison
    inference_times = results_df.groupby('model')['inference_time_per_sample'].mean() * 1000
    inference_times = inference_times.sort_values()
    colors = ['green' if t < 1 else 'orange' if t < 5 else 'red' for t in inference_times.values]

    bars = ax.barh(range(len(inference_times)), inference_times.values, color=colors, alpha=0.8, edgecolor='white')
    ax.set_yticks(range(len(inference_times)))
    ax.set_yticklabels(inference_times.index, fontweight='bold')
    ax.set_xlabel('Inference Time (ms/sample)', fontsize=11, fontweight='bold')
    ax.set_title('Inference Speed Ranking', fontsize=12, fontweight='bold', pad=20)
    ax.set_xscale('log')
    ax.grid(True, alpha=0.3)

def model_distribution(results_df, ax):
    # Plot pie chart of best model distribution
    best_per_dof = results_df.loc[results_df.groupby('dof')['joint_rmse'].idxmin()]
    dof_counts = best_per_dof['model'].value_counts()

    colors = sns.color_palette('husl', len(dof_counts))
    wedges, texts, autotexts = ax.pie(dof_counts.values, labels=dof_counts.index, autopct='%1.1f%%',
                                     startangle=90, colors=colors, textprops={'fontweight': 'bold'})
    ax.set_title('Best Model Distribution Across DOFs', fontsize=12, fontweight='bold', pad=20)

def performance_improvement(results_df, ax):
    # Plot performance improvement with increasing DOF
    for model in results_df['model'].unique():
        model_data = results_df[results_df['model'] == model].sort_values('dof')
        if len(model_data) > 1:
            improvement = (model_data['joint_rmse'].values[:-1] - model_data['joint_rmse'].values[1:]) / model_data['joint_rmse'].values[:-1] * 100
            ax.plot(model_data['dof'].values[1:], improvement, 'o-', label=model, alpha=0.8, linewidth=2)

    ax.set_xlabel('DOF', fontsize=11, fontweight='bold')
    ax.set_ylabel('Performance Change (%)', fontsize=11, fontweight='bold')
    ax.set_title('Performance Change with Increasing DOF', fontsize=12, fontweight='bold', pad=20)
    ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
    ax.grid(True, alpha=0.3)

# NEW VISUALIZATION 1: 3D Accuracy-Speed-DOF Plot
def 3d_plot(results_df, model_colors):
    # 3D plot showing DOF vs Inference Speed vs Joint RMSE
    fig = plt.figure(figsize=(12, 9))
    ax = fig.add_subplot(111, projection='3d')

    for model in results_df['model'].unique():
        model_data = results_df[results_df['model'] == model]
        x = model_data['dof']
        y = model_data['inference_time_per_sample'] * 1000  # Convert to ms
        z = model_data['joint_rmse']

        ax.scatter(x, y, z, c=[model_colors[model]], s=80, alpha=0.8,
                  label=model, edgecolors='white', linewidth=1)

    ax.set_xlabel('DOF', fontsize=11, fontweight='bold')
    ax.set_ylabel('Inference Time (ms)', fontsize=11, fontweight='bold')
    ax.set_zlabel('Joint RMSE', fontsize=11, fontweight='bold')
    ax.set_title('3D Model Performance Tradeoffs\n(DOF vs Speed vs Accuracy)',
                fontsize=14, fontweight='bold', pad=20)

    ax.legend(bbox_to_anchor=(1.15, 1), loc='upper left')
    plt.tight_layout()
    return fig

def pareto_frontier_analysis(results_df, model_colors):
    # Create Pareto frontier analysis for accuracy vs inference speed
    fig, ax = plt.subplots(figsize=(12, 8))

    # Calculate average metrics per model
    model_metrics = results_df.groupby('model').agg({
        'joint_rmse': 'mean',
        'inference_time_per_sample': 'mean'
    }).reset_index()

    # Convert to ms
    model_metrics['inference_time_ms'] = model_metrics['inference_time_per_sample'] * 1000

    # Plot all models
    for _, row in model_metrics.iterrows():
        ax.scatter(row['inference_time_ms'], row['joint_rmse'],
                  s=150, alpha=0.8, color=model_colors[row['model']],
                  edgecolors='white', linewidth=2, label=row['model'])
        ax.annotate(row['model'], (row['inference_time_ms'], row['joint_rmse']),
                   xytext=(8, 8), textcoords='offset points',
                   fontsize=10, fontweight='bold')

    # Find Pareto frontier (minimize both inference time and RMSE)
    points = model_metrics[['inference_time_ms', 'joint_rmse']].values
    pareto_front = []

    for i, point in enumerate(points):
        dominated = False
        for other_point in points:
            if (other_point[0] <= point[0] and other_point[1] <= point[1] and
                (other_point[0] < point[0] or other_point[1] < point[1])):
                dominated = True
                break
        if not dominated:
            pareto_front.append(i)

    # Plot Pareto frontier
    if len(pareto_front) > 1:
        pareto_points = points[pareto_front]
        sorted_indices = np.argsort(pareto_points[:, 0])
        pareto_sorted = pareto_points[sorted_indices]
        ax.plot(pareto_sorted[:, 0], pareto_sorted[:, 1], 'r--',
               linewidth=3, alpha=0.8, label='Pareto Frontier')

    ax.set_xlabel('Inference Time (ms)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Joint RMSE (lower is better)', fontsize=12, fontweight='bold')
    ax.set_title('Pareto Frontier Analysis: Accuracy vs Speed Tradeoff',
                fontsize=14, fontweight='bold', pad=20)
    ax.set_xscale('log')
    ax.grid(True, alpha=0.3)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    return fig

def variance_analysis(results_df, model_colors):
    # Create error bar plots showing model stability across DOF
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # Joint RMSE variance across DOF
    for model in results_df['model'].unique():
        model_data = results_df[results_df['model'] == model]
        dof_stats = model_data.groupby('dof')['joint_rmse'].agg(['mean', 'std']).reset_index()

        ax1.errorbar(dof_stats['dof'], dof_stats['mean'], yerr=dof_stats['std'],
                    marker='o', linewidth=2, markersize=6, alpha=0.8,
                    color=model_colors[model], label=model, capsize=5)

    ax1.set_xlabel('DOF', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Joint RMSE', fontsize=12, fontweight='bold')
    ax1.set_title('Model Stability: RMSE vs DOF', fontsize=12, fontweight='bold', pad=20)
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax1.grid(True, alpha=0.3)

    # Training time variance across DOF
    for model in results_df['model'].unique():
        model_data = results_df[results_df['model'] == model]
        dof_stats = model_data.groupby('dof')['training_time'].agg(['mean', 'std']).reset_index()

        ax2.errorbar(dof_stats['dof'], dof_stats['mean'], yerr=dof_stats['std'],
                    marker='s', linewidth=2, markersize=6, alpha=0.8,
                    color=model_colors[model], label=model, capsize=5)

    ax2.set_xlabel('DOF', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Training Time (s)', fontsize=12, fontweight='bold')
    ax2.set_title('Model Stability: Training Time vs DOF', fontsize=12, fontweight='bold', pad=20)
    ax2.set_yscale('log')
    ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    return fig

def heatmap(results_df):
    # Create high-resolution heatmap with smooth interpolation
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # Prepare data
    models = results_df['model'].unique()
    dofs = sorted(results_df['dof'].unique())

    # Create grid for interpolation
    model_indices = {model: i for i, model in enumerate(models)}
    results_df['model_idx'] = results_df['model'].map(model_indices)

    # Joint RMSE heatmap with interpolation
    pivot_rmse = results_df.pivot_table(values='joint_rmse', index='model', columns='dof', aggfunc='mean')

    # Create smooth interpolation
    xi = np.linspace(0, len(models)-1, len(models)*3)
    yi = np.linspace(min(dofs), max(dofs), len(dofs)*3)
    xi_grid, yi_grid = np.meshgrid(xi, yi)

    # Interpolate
    points = []
    values = []
    for model in models:
        for dof in dofs:
            if not pd.isna(pivot_rmse.loc[model, dof]):
                points.append([model_indices[model], dof])
                values.append(pivot_rmse.loc[model, dof])

    if len(points) > 3:  # Need at least 3 points for interpolation
        zi = griddata(points, values, (xi_grid, yi_grid), method='cubic', fill_value=np.nan)
        im1 = ax1.imshow(zi, extent=[0, len(models)-1, min(dofs), max(dofs)],
                        aspect='auto', origin='lower', cmap='plasma', interpolation='bilinear')
    else:
        im1 = ax1.imshow(pivot_rmse.values, cmap='plasma', aspect='auto')

    ax1.set_xticks(range(len(models)))
    ax1.set_xticklabels(models, rotation=45, ha='right', fontweight='bold')
    ax1.set_ylabel('DOF', fontsize=12, fontweight='bold')
    ax1.set_title('Smooth Joint RMSE Heatmap', fontsize=12, fontweight='bold', pad=20)
    cbar1 = plt.colorbar(im1, ax=ax1, shrink=0.8)
    cbar1.set_label('Joint RMSE', fontweight='bold')

    # Training time heatmap
    pivot_time = results_df.pivot_table(values='training_time', index='model', columns='dof', aggfunc='mean')
    im2 = ax2.imshow(pivot_time.values, cmap='magma', aspect='auto')
    ax2.set_xticks(range(len(pivot_time.columns)))
    ax2.set_xticklabels(pivot_time.columns, fontweight='bold')
    ax2.set_yticks(range(len(pivot_time.index)))
    ax2.set_yticklabels(pivot_time.index, fontweight='bold')
    ax2.set_xlabel('DOF', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Model', fontsize=12, fontweight='bold')
    ax2.set_title('Training Time Heatmap', fontsize=12, fontweight='bold', pad=20)
    cbar2 = plt.colorbar(im2, ax=ax2, shrink=0.8)
    cbar2.set_label('Training Time (s)', fontweight='bold')

    plt.tight_layout()
    return fig

def correlation_analysis(results_df):
    # Create correlation heatmap of all metrics
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # Select numeric columns for correlation
    numeric_cols = ['dof', 'joint_rmse', 'training_time', 'inference_time_per_sample']
    corr_data = results_df[numeric_cols].copy()
    corr_data['inference_time_ms'] = corr_data['inference_time_per_sample'] * 1000
    corr_data = corr_data.drop('inference_time_per_sample', axis=1)

    # Overall correlation matrix
    corr_matrix = corr_data.corr()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

    sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='RdBu_r', center=0,
                square=True, ax=ax1, cbar_kws={'shrink': 0.8})
    ax1.set_title('Overall Metric Correlations', fontsize=12, fontweight='bold', pad=20)

    # Per-model correlation strength
    model_correlations = []
    for model in results_df['model'].unique():
        model_data = results_df[results_df['model'] == model][numeric_cols]
        if len(model_data) > 3:  # Need sufficient data points
            model_corr = abs(model_data.corr()).mean().mean()  # Average absolute correlation
            model_correlations.append({'model': model, 'avg_correlation': model_corr})

    if model_correlations:
        corr_df = pd.DataFrame(model_correlations)
        bars = ax2.bar(corr_df['model'], corr_df['avg_correlation'],
                      color=sns.color_palette('viridis', len(corr_df)), alpha=0.8)
        ax2.set_xlabel('Model', fontsize=12, fontweight='bold')
        ax2.set_ylabel('Average Absolute Correlation', fontsize=12, fontweight='bold')
        ax2.set_title('Model Metric Interdependence', fontsize=12, fontweight='bold', pad=20)
        ax2.set_xticklabels(corr_df['model'], rotation=45, ha='right', fontweight='bold')
        ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    return fig

# MAIN ANALYSIS EXECUTION
if all_test_results:
    print('Enhanced Analytics Dashboard')

    results_df = pd.DataFrame(all_test_results)
    model_colors = colors(results_df)

    # 1. ORIGINAL PLOTS (IMPROVED)
    fig_main = plt.figure(figsize=(20, 12))
    gs = fig_main.add_gridspec(2, 3, hspace=0.35, wspace=0.35)
    fig_main.suptitle('Model Performance Analysis Dashboard', fontsize=16, fontweight='bold', y=0.95)

    # Original 6 plots with improvements
    ax1 = fig_main.add_subplot(gs[0, 0])
    plot_accuracy_vs_speed_tradeoff(results_df, ax1, model_colors)

    ax2 = fig_main.add_subplot(gs[0, 1])
    heatmap(results_df, ax2)

    ax3 = fig_main.add_subplot(gs[0, 2])
    training_distribution(results_df, ax3)

    ax4 = fig_main.add_subplot(gs[1, 0])
    speed_comparison(results_df, ax4)

    ax5 = fig_main.add_subplot(gs[1, 1])
    model_distribution(results_df, ax5)

    ax6 = fig_main.add_subplot(gs[1, 2])
    performance_improvement(results_df, ax6)

    plt.savefig(results_path / 'comprehensive_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()

    # 2. NEW ADVANCED VISUALIZATIONS

    # 3D Tradeoff Analysis
    fig_3d = 3d_plot(results_df, model_colors)
    fig_3d.savefig(results_path / '3d_tradeoff_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()

    # Pareto Frontier Analysis
    fig_pareto = pareto_frontier_analysis(results_df, model_colors)
    fig_pareto.savefig(results_path / 'pareto_frontier_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()

    # Variance Analysis
    fig_variance = variance_analysis(results_df, model_colors)
    fig_variance.savefig(results_path / 'variance_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()

    # Granular Heatmap
    fig_heatmap = heatmap(results_df)
    fig_heatmap.savefig(results_path / 'granular_heatmap.png', dpi=150, bbox_inches='tight')
    plt.show()

    # Correlation Analysis
    fig_corr = correlation_analysis(results_df)
    fig_corr.savefig(results_path / 'correlation_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()


    print(f'\n')
    print('ENHANCED PERFORMANCE STATISTICS')

    print(f'Total experiments: {len(results_df)}')
    print(f'Successful models: {len(results_df[results_df['status'] == 'success'])}')
    print(f'Average accuracy (Joint RMSE): {results_df['joint_rmse'].mean():.4f}')
    print(f'Best overall accuracy: {results_df['joint_rmse'].min():.4f}')
    print(f'Fastest training: {results_df['training_time'].min():.2f}s')
    print(f'Fastest inference: {results_df['inference_time_per_sample'].min()*1000:.2f}ms')

    best_accuracy_model = results_df.loc[results_df['joint_rmse'].idxmin(), 'model']
    fastest_training_model = results_df.loc[results_df['training_time'].idxmin(), 'model']
    fastest_inference_model = results_df.loc[results_df['inference_time_per_sample'].idxmin(), 'model']

    print(f'\nInsights:')
    print(f'Most accurate model: {best_accuracy_model}')
    print(f'Fastest training: {fastest_training_model}')
    print(f'Fastest inference: {fastest_inference_model}')

    results_df['balanced_score'] = (
        (1 / results_df['joint_rmse']) * 0.4 +
        (1 / results_df['training_time']) * 0.3 +
        (1 / results_df['inference_time_per_sample']) * 0.3
    )

    best_balanced = results_df.loc[results_df['balanced_score'].idxmax()]
    print(f'Best balanced model: {best_balanced['model']} (DOF: {best_balanced['dof']})')

    print(f'\nEnhanced analysis complete!')
    print(f'All results and visualizations saved to: {results_path}')

else:
    print('No results data available for analytics')